# 📦 0. Import necessary libraries

In [27]:
import requests
import json
import pandas as pd
import os

from datetime import timedelta
from datetime import datetime
from time import sleep

from db_config import get_engine
from ApiKey import api_key

ModuleNotFoundError: No module named 'db_config'

# ------------------- CONFIG -------------------

# 📦 1 Postgres Database connect

In [ ]:
engine = get_engine()

ModuleNotFoundError: No module named 'config'

# 🔐 2. Configure API key, headers, players name and url

In [14]:
# We create the headers that PUBG requires to authenticate and specify the format
HEADERS = {
    "Authorization": f"Bearer {api_key}",
    "Accept": "application/vnd.api+json"
}

# URL base to the PUBG API
base_url = "https://api.pubg.com/shards/steam"

NameError: name 'api_key' is not defined

# ------------------- EXTRACT -------------------

# 🎮 3. Find player ID

In [25]:
def searchPlayer (player_name):
    # Endpoint to find player for name
    url = f"{base_url}/players?filter[playerNames]={player_name}"
    
    # Send the GET request to the API with authorization headers
    response = requests.get(url, headers=HEADERS)

    # Raise an exception if the request failed (e.g., 404 or 500)
    response.raise_for_status()    

    #Save the information and transform with Json library and return the player id
    data = response.json()
    return data["data"][0]["id"]

# 🎯 4. Obtain matches from the player

In [24]:
def find_match_players(player_id):
    # Endpoint to obtain data of the player from id
    url = f"{base_url}/players/{player_id}"

    # Send the GET request to the API with authorization headers
    response = requests.get(url, headers=HEADERS)
    
    # Raise an exception if the request failed (e.g., 404 or 500)
    response.raise_for_status()  

    # Returns all of matches from the player
    return response.json()["data"]["relationships"]["matches"]["data"]

# 📊 5. Obtain match details

In [23]:
def extract_match_details(match_id):
    # Construct the API endpoint URL using the match ID
    url = f"{base_url}/matches/{match_id}"
    
    # Send the GET request to the API with authorization headers
    response = requests.get(url, headers=HEADERS)
    
    # Raise an exception if the request failed (e.g., 404 or 500)
    response.raise_for_status()
    
    # Return the JSON response containing match details
    return response.json()

# 📊 6. Obtain telemetry details

In [ ]:
def download_telemetry(match_id):
    url = f"{base_url}/matches/{match_id}"
    response = requests.get(url, headers=HEADERS)
    response.raise_for_status()
    match_data = response.json()

    # Obtener asset ID de telemetría
    asset_id = match_data['data']['relationships']['assets']['data'][0]['id']

    # Buscar el asset en el array 'included'
    asset = next((item for item in match_data['included'] if item['type'] == 'asset' and item['id'] == asset_id), None)
    if asset is None:
        print("❌ Telemetry asset not found.")
        return match_data

    telemetry_url = asset['attributes']['URL']
    
    tele_response = requests.get(telemetry_url, headers={"Accept-Encoding": "gzip"})
    tele_response.raise_for_status()

    os.makedirs("telemetry", exist_ok=True)
    filepath = f"telemetry/{match_id}_telemetry.json"
    with open(filepath, "wb") as f:
        f.write(tele_response.content)

    print(f"✅ Telemetría guardada en {filepath}")

download_telemetry(find_match_players(searchPlayer(player_name))[0]['id'])

# ------------------- TRANSFORM -------------------

In [22]:
def match_info(match_data):
    data = match_data['data']['attributes']
    rows = []
    rows.append({
            "gameMode":data['gameMode'],
            "mapName": data["mapName"],
            "createdAt": data['createdAt'],
            "isCustomMatch": data["isCustomMatch"],
            "matchType": data["matchType"],
            "seasonState": data["seasonState"]
    })
        
    return pd.DataFrame(rows)

In [21]:
def rooster_info(match_data):
    rosters = match_data['data']['relationships']['rosters']['data']
    rows = []
    for r in rosters:
        rows.append({
            "id": r['id']
        })
        
    return pd.DataFrame(rows)

In [20]:
def player_match_info(match_data):
    participants = [x for x in match_data['included'] if x['type'] == 'participant']
    rows = []
    for p in participants:
        stats = p['attributes']['stats']
        rows.append({
            "name": stats['name'],
            "playerId": stats["playerId"],
            "winPlace": stats["winPlace"],
            "kills": stats['kills'],
            "headshotKills": stats["headshotKills"],
            "longestKill": stats["longestKill"],
            "killPlace": stats["killPlace"],
            "DBNOs": stats["DBNOs"],
            "assists": stats["assists"],
            "damageDealt": stats['damageDealt'],
            "winPlace": stats['winPlace'],
            "timeSurvived": str(pd.to_timedelta(stats['timeSurvived'], unit='s')).split()[-1],
            "heals": stats["heals"],
            "boosts": stats["boosts"],
            "deathType": stats["deathType"],
            "walkDistance": stats["walkDistance"],
            "rideDistance": stats["rideDistance"],
            "swimDistance": stats["swimDistance"],
            "vehicleDestroys": stats["vehicleDestroys"]
        })
    pd.set_option('display.max_rows', None)
    return pd.DataFrame(rows).sort_values(by=['winPlace', 'kills'], ascending=[True, False]).reset_index(drop=True)

# ------------------- LOAD CSV------------------- 

In [19]:
def save_to_csv(df, filename="data/match_data.csv"):
    os.makedirs(os.path.dirname(filename), exist_ok=True)
    df.to_csv(filename, index=False)

In [18]:
def run_etl(player_name, max_matches=3):
    print(f"Obteniendo ID para jugador: {player_name}")
    player_id = searchPlayer(player_name)
    
    print("Obteniendo lista de partidas...")
    matches = find_match_players(player_id)[:max_matches]
    sleep(1)

    # Inicializamos los tres dataframes
    all_data_match = pd.DataFrame()
    all_data_rooster = pd.DataFrame()
    all_data_player = pd.DataFrame()
    
    for match in matches:
        match_id = match['id']
        print(f"Procesando partida: {match_id}")
        match_data = extract_match_details(match_id)
        sleep(1)

        # Match info
        df_match = match_info(match_data)
        df_match['match_id'] = match_id
        all_data_match = pd.concat([all_data_match, df_match], ignore_index=True)
        sleep(1)
        
        # Rooster info
        df_rooster = rooster_info(match_data)
        df_rooster['match_id'] = match_id
        all_data_rooster = pd.concat([all_data_rooster, df_rooster], ignore_index=True)
        sleep(1)
        
        # Player info
        df_player = player_match_info(match_data)
        df_player['match_id'] = match_id
        all_data_player = pd.concat([all_data_player, df_player], ignore_index=True)

        sleep(1)

    # Timestamp y nombres de archivo
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    base_path = "data"

    player_file = f"{base_path}/{player_name}_players_{timestamp}.csv"
    rooster_file = f"{base_path}/{player_name}_roosters_{timestamp}.csv"
    match_file = f"{base_path}/{player_name}_matches_{timestamp}.csv"

    print("Guardando archivos CSV...")
    save_to_csv(all_data_player, filename=player_file)
    save_to_csv(all_data_rooster, filename=rooster_file)
    save_to_csv(all_data_match, filename=match_file)

    print("✅ ETL completado. Archivos generados:")
    print(f" - {player_file}")
    print(f" - {rooster_file}")
    print(f" - {match_file}")

    return {
        "player_csv": player_file,
        "rooster_csv": rooster_file,
        "match_csv": match_file
    }

# ------------------- LOAD POSTGRES------------------- 

In [17]:
def upload_csv_to_postgres(csv_file_path, table_name, key_column="match_id"):
    try:
        # Leer el archivo CSV
        df = pd.read_csv(csv_file_path)

        # Obtener los match_id ya existentes en la tabla
        existing_ids_query = f"""
            SELECT {key_column} FROM pubg_games.{table_name}
        """
        existing_ids = pd.read_sql(existing_ids_query, engine)
        existing_ids_set = set(existing_ids[key_column].astype(str))

        # Convertimos el match_id a string (por seguridad) y filtramos duplicados
        df[key_column] = df[key_column].astype(str)
        df_filtered = df[~df[key_column].isin(existing_ids_set)]

        if df_filtered.empty:
            print(f"⚠️ Todos los registros del archivo ya existen en la tabla '{table_name}'. No se insertó nada.")
        else:
            df_filtered.to_sql(table_name, engine, if_exists="append", index=False, schema="pubg_games")
            print(f"✅ Insertados {len(df_filtered)} registros nuevos en '{table_name}'.")

    except Exception as e:
        print(f"❌ Error al subir a PostgreSQL: {e}")

In [16]:
def load_postgres(csv_file_path, table_name):
    print(f"archivo: '{csv_file_path}'")
    print(f"tabla: '{table_name}'")
    
    upload_csv_to_postgres(csv_file_path, table_name, key_column="match_id")
    print("✅ Upload completo.")

# ------------------- MAIN PIPELINE -------------------

In [26]:
csv_paths = run_etl("KaDiz-", max_matches=3)

load_postgres(csv_paths["player_csv"], table_name="players", )
load_postgres(csv_paths["rooster_csv"], table_name="roosters")
load_postgres(csv_paths["match_csv"], table_name="matches")

Obteniendo ID para jugador: KaDiz-


NameError: name 'base_url' is not defined